# Da best heart disease classifier in town
- 13516083 / Abram Perdanaputra
- 13516090 / Timothy Thamrin Andrew Hamonangan Sihombing
- 13516093 / Muhammad Farhan
- 13516153 / Dimas Aditia Pratikto
- 13516155 / Restu Wahyu Kartiko

In [21]:
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

## Read data

In [72]:
heart_disease = {}
heart_disease['columns'] = [
    'Age', 
    'Sex', 
    'Pain type', 
    'Blood pressure', 
    'Serum cholesterol', 
    'Fasting blood sugar > 120mg/dl', 
    'Resting ECG', 
    'Max heart rate achieved', 
    'exercise induced agina', 
    'ST depression induced by exercise relative to rest', 
    'Peak exercise ST segment', 
    'Number of major vessels colored by flourosopy', 
    'Thal', 
    'Diagnosis'
]
heart_disease['train'] = pd.read_csv('../data/tubes2_HeartDisease_train.csv', skiprows=[0], names=heart_disease['columns'])
heart_disease['test'] = pd.read_csv('../data/tubes2_HeartDisease_test.csv', skiprows=[0], names=heart_disease['columns'])

## Exploratory data analysis

In [73]:
heart_disease['train'].head()

,Age,Sex,Pain type,Blood pressure,Serum cholesterol,Fasting blood sugar > 120mg/dl,Resting ECG,Max heart rate achieved,exercise induced agina,ST depression induced by exercise relative to rest,Peak exercise ST segment,Number of major vessels colored by flourosopy,Thal,Diagnosis
0,54,1,4,125,216,0,0,140,0,0,?,?,?,1
1,55,1,4,158,217,0,0,110,1,2.5,2,?,?,1
2,54,0,3,135,304,1,0,170,0,0,1,0,3,0
3,48,0,3,120,195,0,0,125,0,0,?,?,?,0
4,50,1,4,120,0,0,1,156,1,0,1,?,6,3


Check for NULL values

In [111]:
null = [[], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
a = heart_disease['train'].isnull().sum()
null[0] = [a[i] for i in range(len(a))]
for i, col in enumerate(heart_disease['train'].columns):
    null_train = heart_disease['train'][heart_disease['train'][col].astype(str) == '?']
    null[0][i] += int(null_train.shape[0])
    null[1][i] += null[0][i] / heart_disease['train'].shape[0] * 100
    
null_df = pd.DataFrame(null, columns=heart_disease['columns'], index=['Num', '%'], dtype=int)
null_df

,Age,Sex,Pain type,Blood pressure,Serum cholesterol,Fasting blood sugar > 120mg/dl,Resting ECG,Max heart rate achieved,exercise induced agina,ST depression induced by exercise relative to rest,Peak exercise ST segment,Number of major vessels colored by flourosopy,Thal,Diagnosis
Num,0,0,0,47,24,78,2,44,44,49,262,514,408,0
%,0,0,0,6,3,10,0,5,5,6,33,65,52,0


- <b>Univariate data visualization</b>

- <b>Bivariate data visualization</b>

- <b>Multivariate data visualization</b>

- <b>Dimensionality reduction</b>

## Data preprocessing

2 jenis:
1. numeric
    - preprocess: normalize
    - null: mean or median, or buat model yang ditrain di datayang ganull, terus predict null
2. categorical
    1. Nominal: urutan gangaruh (fakultas)
         - preprocess: one hot encoding
    2. ordinal: categorical yang urutan ngaruh (rating)
         - preprocess: label encoding pd.get dummies

## Experiment